In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
    print("CUDA version:", torch.version.cuda)


CUDA available: True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Number of GPUs: 1
CUDA version: 12.1


In [2]:

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
    print("CUDA version:", torch.version.cuda)


CUDA available: True
GPU name: NVIDIA GeForce RTX 4060 Laptop GPU
Number of GPUs: 1
CUDA version: 12.1


In [3]:
# =============================
# Step 0: Install / Import
# =============================
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms, models
from PIL import Image
import copy

In [4]:
# =============================
# Step 1: Custom Dataset
# =============================
class CropDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.images = []
        self.labels = []
        self.class_names = []

        crop_folders = [f for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
        crop_folders.sort()
        self.class_names = crop_folders
        self.class_to_idx = {name: idx for idx, name in enumerate(self.class_names)}

        for crop_name in crop_folders:
            raw_path = os.path.join(root_dir, crop_name, 'raw')
            image_files = glob.glob(os.path.join(raw_path, '*'))
            for img_path in image_files:
                self.images.append(img_path)
                self.labels.append(self.class_to_idx[crop_name])

        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# =============================
# Step 2: Transformations
# =============================
# CHANGED: Simplified the training augmentations to a more standard, less aggressive set.
# REASON: Overly aggressive augmentations can create unrealistic images and hinder learning.
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(), # A strong, modern auto-augmentation policy
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    # CenterCrop is not strictly necessary if you resize to the exact dimensions
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [6]:
# =============================
# Step 3: Load Dataset
# =============================
## FIX: Create two separate dataset instances to correctly apply transforms.
dataset_path = "Crop_Image_Data/Crop_Dataset"

# Create a dataset for training with training transforms
dataset_for_splitting = CropDataset(dataset_path, transform=train_transforms)
print(f"✅ Found {len(dataset_for_splitting.class_names)} crop categories")
for idx, name in enumerate(dataset_for_splitting.class_names):
    print(f"{idx}: {name}")

# Create the validation dataset by deep copying the first one and assigning validation transforms
val_dataset_full = copy.deepcopy(dataset_for_splitting)
val_dataset_full.transform = val_transforms

# Generate indices for splitting
dataset_size = len(dataset_for_splitting)
indices = list(range(dataset_size))
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

# Perform the split using torch.Generator for reproducibility
generator = torch.Generator().manual_seed(42)
train_indices, val_indices = random_split(indices, [train_size, val_size], generator=generator)

# Use the indices to create Subset objects from the correctly transformed datasets
train_dataset = Subset(dataset_for_splitting, train_indices)
val_dataset = Subset(val_dataset_full, val_indices)


batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


✅ Found 140 crop categories
0: Aji pepper plant
1: Almonds plant
2: Amaranth plant
3: Apples plant
4: Artichoke plant
5: Avocados plant
6: Açaí plant
7: Bananas plant
8: Barley plant
9: Beets plant
10: Black pepper plant
11: Blueberries plant
12: Bok choy plant
13: Brazil nuts plant
14: Broccoli plant
15: Brussels sprout plant
16: Buckwheat plant
17: Cabbages and other brassicas plant
18: Camucamu plant
19: Carrots and turnips plant
20: Cashew nuts plant
21: Cassava plant
22: Cauliflower plant
23: Celery plant
24: Cherimoya plant
25: Cherry plant
26: Chestnuts plant
27: Chickpeas plant
28: Chili peppers and green peppers plant
29: Cinnamon plant
30: Cloves plant
31: Cocoa beans plant
32: Coconuts plant
33: Coffee (green) plant
34: Collards plant
35: Cotton lint plant
36: Cranberries plant
37: Cucumbers and gherkins plant
38: Dates plant
39: Dry beans plant
40: Dry peas plant
41: Durian plant
42: Eggplants (Aubergines) plant
43: Endive plant
44: Fava bean plant
45: Figs plant
46: Flax f

In [7]:
# =============================
# Step 4: Model Setup
# =============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("\n🚀 Training will run on:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Replace final FC layer
num_classes = len(dataset_for_splitting.class_names)
model.fc = nn.Sequential(
    ## IMPROVEMENT: Slightly higher dropout as a stronger regularizer.
    nn.Dropout(0.6),
    nn.Linear(model.fc.in_features, num_classes)
)
model = model.to(device)

# Loss function with label smoothing remains a good choice
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)


🚀 Training will run on: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [8]:
# =============================
# Step 5: Training Loop (with Differential Learning Rates)
# =============================
EPOCHS = 30
best_val_acc = 0.0
patience = 8
early_stop_counter = 0

print("\n📢 Starting training on device:", device)
print(f"Total train images: {len(train_dataset)}, Total val images: {len(val_dataset)}")
print("="*30)

## IMPROVEMENT: Implement Differential Learning Rates for more effective fine-tuning.
# The "head" (our new classifier) gets a higher learning rate.
# The "body" (pretrained backbone) gets a very low learning rate.
optimizer = optim.Adam([
    {'params': model.conv1.parameters(), 'lr': 1e-5},
    {'params': model.bn1.parameters(), 'lr': 1e-5},
    {'params': model.relu.parameters(), 'lr': 1e-5},
    {'params': model.maxpool.parameters(), 'lr': 1e-5},
    {'params': model.layer1.parameters(), 'lr': 1e-5},
    {'params': model.layer2.parameters(), 'lr': 2e-5},
    {'params': model.layer3.parameters(), 'lr': 5e-5},
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.avgpool.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(), 'lr': 1e-3} # Highest LR for the head
], lr=1e-3, weight_decay=5e-4) # Increased weight_decay for stronger regularization

# A good scheduler is crucial
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    train_loss = running_loss / total_train
    train_acc = correct_train / total_train

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()

    val_loss /= total_val
    val_acc = correct_val / total_val

    # Get current learning rate for the head (the most important one to track)
    current_lr = optimizer.param_groups[-1]['lr']
    
    print(f"Epoch [{epoch+1:02d}/{EPOCHS}] "
          f"LR: {current_lr:.6f} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    # Scheduler step after the epoch
    scheduler.step()

    # Save best model and Early Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "resnet18_crop_best.pth")
        print(f"💾 Best model updated at epoch {epoch+1} with Val Acc: {val_acc:.4f}")
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("⏹️ Early stopping triggered.")
            break


📢 Starting training on device: cuda
Total train images: 30832, Total val images: 7708
Epoch [01/30] LR: 0.001000 | Train Loss: 4.1059, Acc: 0.1868 | Val Loss: 3.0008, Acc: 0.4267
💾 Best model updated at epoch 1 with Val Acc: 0.4267
Epoch [02/30] LR: 0.000997 | Train Loss: 3.3807, Acc: 0.3305 | Val Loss: 2.7808, Acc: 0.4949
💾 Best model updated at epoch 2 with Val Acc: 0.4949
Epoch [03/30] LR: 0.000989 | Train Loss: 3.0823, Acc: 0.4014 | Val Loss: 2.6472, Acc: 0.5381
💾 Best model updated at epoch 3 with Val Acc: 0.5381
Epoch [04/30] LR: 0.000976 | Train Loss: 2.8798, Acc: 0.4581 | Val Loss: 2.6041, Acc: 0.5505
💾 Best model updated at epoch 4 with Val Acc: 0.5505
Epoch [05/30] LR: 0.000957 | Train Loss: 2.7019, Acc: 0.5052 | Val Loss: 2.5643, Acc: 0.5679
💾 Best model updated at epoch 5 with Val Acc: 0.5679
Epoch [06/30] LR: 0.000933 | Train Loss: 2.5541, Acc: 0.5453 | Val Loss: 2.5304, Acc: 0.5817
💾 Best model updated at epoch 6 with Val Acc: 0.5817


KeyboardInterrupt: 

In [ ]:
# =============================
# Step 6: Save Final Model + ONNX (No changes needed)
# =============================
# Load the best performing model before saving the final version
print(f"\nLoading best model with accuracy: {best_val_acc:.4f}")
model.load_state_dict(torch.load("resnet18_crop_best.pth"))

final_model_path = "resnet18_crop_final.pth"
torch.save(model.state_dict(), final_model_path)
print(f"✅ Final model saved: {final_model_path}")

onnx_model_path = "resnet18_crop.onnx"
dummy_input = torch.randn(1, 3, 224, 224, device=device)
model.eval() # Set model to evaluation mode for ONNX export
torch.onnx.export(
    model,
    dummy_input,
    onnx_model_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)
print(f"✅ Model exported to ONNX format: {onnx_model_path}")